In [1]:
# %pip install nltk
%pip install pyarrow
%pip install fastparquet

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
df1 = pd.read_csv("submission_queries.csv")
df1

,query
0,экстрасенсы битва сильнейших 2 сезон
1,битва экстрасенсов 2
2,Экстрасенсы
3,битва сильнейших 19 сезон
4,битва экстрасенсов 20 сезон
...,...
1995,ПОЛИКЛИНИКА в этом ведре и есть счастье ?
1996,Как избавиться от страхов и начать бизнес ?
1997,Как и где продавать аромасвечи ?
1998,Ответы на Ваши комментарии! Аромасвечи


In [39]:
# Оставим только слова
df = df1['query']

rus_alph = set(list('йцукенгшщзхъфывапролджэёячсмитьбю0123456789qwertyuiopasdfghjklzxcvbnm'))
drop_not_letters = lambda s: ''.join(list(filter(lambda c: c in rus_alph or c == ' ', s)))

df = [drop_not_letters(comm.lower()) for comm in df]

# уберем пустые комменты
df = list(filter(lambda comm: len(comm) > 0, df))
print(len(df))

df[:10]

2000


['экстрасенсы битва сильнейших 2 сезон',
 'битва экстрасенсов 2',
 'экстрасенсы',
 'битва сильнейших 19 сезон',
 'битва экстрасенсов 20 сезон',
 'битва экстрасенсов',
 'битва сильнейших 19 сезон',
 'экстрасенсы битва сильнейших',
 'звезды в дубае',
 'выжить в дубае звездам прислали верблюжий навоз']

In [166]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords', quiet=True)

ru_stopwords = stopwords.words('russian')
print(len(ru_stopwords))

151


In [38]:
ru_stopwords = set(ru_stopwords)

def preprocess_text(text: str) -> str:
    '''text = text.lower()
    is_allowed_char = lambda c: c.isalpha() or c == ' '
    text = ''.join(list(filter(is_allowed_char, text)))'''

    is_stopword = lambda word: word not in ru_stopwords
    text = ' '.join(list(filter(is_stopword, text.split())))
    return text


texts = [preprocess_text(text) for text in df]

texts[:10]

['экстрасенсы битва сильнейших 2 сезон',
 'битва экстрасенсов 2',
 'экстрасенсы',
 'битва сильнейших 19 сезон',
 'битва экстрасенсов 20 сезон',
 'битва экстрасенсов',
 'битва сильнейших 19 сезон',
 'экстрасенсы битва сильнейших',
 'звезды дубае',
 'выжить дубае звездам прислали верблюжий навоз']

In [20]:
from w_dict_auto import words_dictionary_auto as W1
from w_dict_manual import words_dictionary_manual as W2


In [ ]:
answer  = [text.split(' ') for text in df]
print(answer)

In [16]:
from zapros_analysis import highlighting_keywords

In [6]:
features_nov = pd.read_parquet('features_nov.parquet',engine='fastparquet')

In [44]:
features_vid = features_nov[features_nov['video_id'] == 'video_30396638']
features_vid

,video_id,report_date,v_channel_reg_datetime,v_channel_type,v_category,v_pub_datetime,total_comments,v_year_views,v_month_views,v_week_views,v_day_views,v_likes,v_dislikes,v_duration,v_cr_click_like_7_days,v_cr_click_dislike_7_days,v_cr_click_vtop_7_days,v_cr_click_long_view_7_days,v_cr_click_comment_7_days,v_cr_click_like_30_days,v_cr_click_dislike_30_days,v_cr_click_vtop_30_days,v_cr_click_long_view_30_days,v_cr_click_comment_30_days,v_cr_click_like_1_days,v_cr_click_dislike_1_days,v_cr_click_vtop_1_days,v_cr_click_long_view_1_days,v_cr_click_comment_1_days
12610504,video_30396638,2023-11-02,2021-03-18 08:06:30,ТВ И СМИ,Телепередачи,2023-07-02 19:00:13,190,339734,26005,5594,650,753,29,3043000,0.001966,0.000179,0.000715,0.58956,0.0,0.0025,0.000038,0.000692,0.592924,0.000077,0.001538,0.001538,0.0,0.575385,0.0


In [171]:
from functools import cmp_to_key

map_vid_count = dict()
videos_critery_crutosty = dict()


def critery_crutosty(vid):
    try:
        otv = 0
        vid_np = np.array(features_nov[features_nov['video_id'] == vid])[0]
        otv += vid_np[24] + vid_np[25] + vid_np[26] + vid_np[27] + vid_np[28]    
        otv += 0.75 * (vid_np[14] + vid_np[15] + vid_np[16] + vid_np[17] + vid_np[18])
        otv += 0.5 * (vid_np[19] + vid_np[20] + vid_np[21] + vid_np[22] + vid_np[23])
        return otv
    except Exception:
        return 1


def comparator(item1, item2):
    mvc1 = map_vid_count[item1]
    mvc2 = map_vid_count[item2]
    if mvc1 < mvc2:
        return -1
    elif mvc1 > mvc2:
        return 1
    else:    
        cc1 = videos_critery_crutosty[item1]
        cc2 = videos_critery_crutosty[item2]
        if cc1 < cc2:
            return -1
        elif cc1 > cc2:
            return 1
        else:
            return 0


def rec_vids_from_queries(query):
    hkq = highlighting_keywords(query)
    for el in hkq:
        if el in W1 and el in W2:
            for vid in W1[el]:
                if vid in map_vid_count:
                    map_vid_count[vid] += 1
                else:
                    map_vid_count[vid] = 1
            for vid in W2[el]:
                if vid not in W1[el]:
                    if vid in map_vid_count:
                        map_vid_count[vid] += 1
                    else:
                        map_vid_count[vid] = 1
        elif el in W1:
            for vid in W1[el]:
                if vid in map_vid_count:
                    map_vid_count[vid] += 1
                else:
                    map_vid_count[vid] = 1
        elif el in W2:
            for vid in W2[el]:
                if vid in map_vid_count:
                    map_vid_count[vid] += 1
                else:
                    map_vid_count[vid] = 1
    
    # sort_data = sorted(map_vid_count.items(), key=cmp_to_key(comparator))
    # print(sort_data)
    # return list(map(lambda x: x[0], sort_data))[:5]
    
    vids = []
    sum_len = 0
    for i in range(len(hkq), 1, -1):
        for vid in map_vid_count.keys():
            if map_vid_count[vid] == i:
                vids.append(vid)
                videos_critery_crutosty[vid] = critery_crutosty(vid)
                sum_len += 1
            if sum_len >= 10:
                break
        if sum_len >= 5:
            break
    
    vids = sorted(vids, key=cmp_to_key(comparator))
    return vids[:5]
    



In [173]:
zapros = "экстрасенсы битва сильнейших 2 сезон"
print(rec_vids_from_queries(zapros))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['video_23236855', 'video_8794018', 'video_23927899', 'video_32733092', 'video_617238']


In [174]:
# отдельная функция преобразования в файл

In [175]:
tst = np.array(pd.read_csv('test_dataset_submission_queries.csv'))
tst

array([['Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия'],
       ['битва сильнейших экстрасенсов 2023\\nбитва сильнейших экстрасенсов 2023 смотреть'],
       ['Экстрасенсы. Битва сильнейших, 4 выпуск'],
       ...,
       ['смотреть мужское женское 2019'],
       ['шоу вована и лексуса'],
       ['мужкое женское выпуск от 04 06 2021']], dtype=object)

In [ ]:
sub = open('submition_2.txt', 'w')
sub.write("query,video_id\n")

for num, el in enumerate(tst):
    for vid in rec_vids_from_queries(el[0]):
        sub.write(f"{el[0]},{vid}\n")
        print(f"{el[0]},{vid}\n")
sub.close()       

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия,video_32276606

Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия,video_33931132

Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия,video_3810119

Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия,video_5410760

Битва сильнейших экстрасенсов 2023 смотреть | Экстрасенсы битва сильнейших 35, 36 серия,video_18100862



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_11978458

битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_18493556

битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_9467797

битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_8394567

битва сильнейших экстрасенсов 2023\nбитва сильнейших экстрасенсов 2023 смотреть,video_2829093



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Копенгаген манчестер юнайтед,video_1543419

Копенгаген манчестер юнайтед,video_7028036

Копенгаген манчестер юнайтед,video_11424806

Копенгаген манчестер юнайтед,video_10988430

Копенгаген манчестер юнайтед,video_9150521



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


мужское женское 14.11.2023,video_1543419

мужское женское 14.11.2023,video_7028036

мужское женское 14.11.2023,video_11424806

мужское женское 14.11.2023,video_10988430

мужское женское 14.11.2023,video_9150521



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
